# Linking Writing Processes to Writing Quality

## Library

In [1]:
import os
import sys
import gc
import random
from glob import glob
from pathlib import Path
from collections import Counter, defaultdict
import re
import copy
import ctypes
libc = ctypes.CDLL("libc.so.6")  # clear the memory

import numpy as np
import pandas as pd
import torch
from scipy.stats import skew, kurtosis

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor

# model
import optuna
import lightgbm as lgbm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor 
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingRegressor

In [2]:
# /kaggle/working配下にディレクトリを作成
!mkdir models
!mkdir oof

## Configurations

In [3]:
pd.set_option("display.max_columns", 3000)

In [4]:
class Config:
    VERSION = 1
    COMPETITION = "linking-writing-processes-to-writing-quality"
    DATA_PATH = os.path.join("/", "kaggle", "input", COMPETITION)
    OOF_DATA_PATH = Path("./oof")
    MODEL_DATA_PATH = Path("./models")
    
    USES_GPU = False
    
    seed = 42
    
    """
    > Optuna options
    """
    USES_OPTUNA_TUNING = False
    
    test_size_optuna = 0.2
    num_trials_optuna = 80
    num_trials_early_stopping = 40
    
    """
    > Modeling options
    """
    target_col = "score"
    metric = "rmse"
    
    num_boost_rounds = 50500
    early_stopping_round = 250
    verbose = 100
    
    num_folds = 10
    
    if USES_GPU:
        regression_lgb_params["device"] = "gpu"
        regression_xgb_params["tree_method"] = "gpu_hist"
        regression_xgb_params["predictor"] = "gpu_predictor"
        regression_cat_params["task_type"] = "GPU"

### Set seed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # CUDAのseedを発生するためにtorchを使用
    torch.manual_seed(seed)
    if Config.USES_GPU:
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(Config.seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


## Load data

In [6]:
train_df = pd.read_csv(os.path.join(Config.DATA_PATH, "train_logs.csv"))
scores_df = pd.read_csv(os.path.join(Config.DATA_PATH, "train_scores.csv"))

test_df = pd.read_csv(os.path.join(Config.DATA_PATH, "test_logs.csv"))

In [7]:
train_df

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,NoChange,1031,240
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,NoChange,1031,240
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240


In [8]:
scores_df

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0
...,...,...
2466,ffb8c745,3.5
2467,ffbef7e5,4.0
2468,ffccd6fd,1.5
2469,ffec5b38,5.0


## Feature engineering

### Essay Constructor

- MASKされてバラバラになったエッセイを再構築する！
- https://www.kaggle.com/code/yuriao/fast-essay-constructor

In [9]:
class EssayConstructor:
    def get_essays(self, df):
        # 必要なカラムのみ抽出
        text_input_df = copy.deepcopy(
            df[["id", "activity", "cursor_position", "text_change"]]
        )
        text_input_df = text_input_df[text_input_df.activity != "Nonproduction"] # Nonproductionの除外
        
        tqdm.pandas()
        essay = text_input_df.groupby("id")[
            ["activity", "cursor_position", "text_change"]
        ].progress_apply(lambda v: self._process_inputs(v))
        
        essay_df = essay.to_frame().reset_index()
        essay_df.columns = ["id", "essay"]
        
        return essay_df
        
    def _process_inputs(self, currTextInput):
        """データのテキスト入力イベントの値からエッセイを復元
        
        Args:
            currTextInput: train_logsからinput系のカラムを抽出したもの
                ["activity", "cursor_position", "text_change", "id"]
        """
        # Where the essay content will be stored
        essayText = ""
        
        # Produces the essay
        for Input in currTextInput.values:
            activity = Input[0]
            
            if activity == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                
            elif activity == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                
            elif activity == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                
            elif "M" in activity:
                # Gets rid of the "Move from to" text
                croppedTxt = activity[10:]              
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')              
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]              
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            
            else:
                # If activity = input
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        
        return essayText

### Preprocessor

In [10]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

In [11]:
class Preprocessor:
    def __init__(self, seed):
        """コンストラクタ
        
        Args:
            seed: シード値
        """
        self.seed = seed
        
        self.activities = ["Input", "Remove/Cut", "Nonproduction", "Replace", "Paste"]
        self.events = [
            "q",
            "Space",
            "Backspace",
            "Shift",
            "ArrowRight",
            "Leftclick",
            "ArrowLeft",
            ".",
            ",",
            "ArrowDown",
            "ArrowUp",
            "Enter",
            "CapsLock",
            "'",
            "Delete",
            "Unidentified",
        ]
        self.text_changes_dict = {
            "q": "q",
            " ": "space",
            "NoChange": "NoChange",
            ".": "full_stop",
            ",": "comma",
            "\n": "newline",
            "'": "single_quote",
            '"': "double_quote",
            "-": "dash",
            "?": "question_mark",
            ";": "semicolon",
            "=": "equals",
            "/": "slash",
            "\\": "double_backslash",
            ":": "colon",
        }
        self.punctuations = ['"', ".", ",", '"', "-", ";", ":", "?", "!", "<", ">", "/", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "+"]
        
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 70, 100]
        
        self.idf = defaultdict(float)
        
        self.essay_constructor = EssayConstructor()
    
    def make_feats(self, df: pd.DataFrame):
        """特徴量の作成
        
        Args:
            df (pd.DataFrame): 入力データ
        """
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({"id": df["id"].unique().tolist()})
        # TODO: Argsとする
        essay_df = self.essay_constructor.get_essays(df)
        
        # Essay統計量
        essay_agg_df = self._count_essay_str(essay_df)
        feats = feats.merge(essay_agg_df, on="id", how="left")
        
        word_agg_df = self._calc_word_aggregations(essay_df)
        feats = feats.merge(word_agg_df, on="id", how="left")
        
        sentence_agg_df = self._calc_sentence_aggregations(essay_df)
        feats = feats.merge(sentence_agg_df, on="id", how="left")
        
        paragraph_agg_df = self._calc_paragraph_aggregations(essay_df)
        feats = feats.merge(paragraph_agg_df, on="id", how="left")
        
        product_to_keys_df = self._product_to_keys(df, essay_df)
        feats = feats.merge(product_to_keys_df, on="id", how="left")
        
        mouse_event_df = self._create_mouse_event_feature(df)
        feats = feats.merge(mouse_event_df, on="id", how="left")
        
        shortcut_event_df = self._create_shortcut_event_feature(df)
        feats = feats.merge(shortcut_event_df, on="id", how="left")
        
        # 統計量
        tmp_df = self._count_activity(df=df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        tmp_df = self._count_text_change(df=df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        tmp_df = self._count_punctuations(df=df, colname="down_event")
        feats = pd.concat([feats, tmp_df], axis=1)
        
        tmp_df = self._calc_p_bursts(df=df)
        feats = pd.merge(feats, tmp_df, on="id", how="left")
        
        tmp_df = self._calc_r_bursts(df=df)
        feats = pd.merge(feats, tmp_df, on="id", how="left")
        
        tmp_df = self._calc_keys_pressed_per_second(df=df)
        feats = pd.merge(feats, tmp_df, on="id", how="left")
        
        # space features
        tmp_df = self._create_space_features(df)
        feats = feats.merge(tmp_df, on="id", how="left")
        
        # shifted features
        # 特定の時間や行動間のギャップを計算
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f"up_time_shift{gap}"] = df.groupby("id")["up_time"].shift(gap)
            df[f"action_time_gap{gap}"] = df["down_time"] - df[f"up_time_shift{gap}"]
            
        df.drop(columns=[f"up_time_shift{gap}" for gap in self.gaps], inplace=True)
        
        # cursor position shift
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f"cursor_position_shift{gap}"] = df.groupby("id")["cursor_position"].shift(gap)
            df[f"cursor_position_change{gap}"] = df["cursor_position"] - df[f"cursor_position_shift{gap}"]
            df[f"cursor_position_abs_change{gap}"] = np.abs(df[f"cursor_position_change{gap}"])
        
        df.drop(columns=[f"cursor_position_shift{gap}" for gap in self.gaps], inplace=True)
        
        # word count shift
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f"word_count_shift{gap}"] = df.groupby("id")["word_count"].shift(gap)
            df[f"word_count_change{gap}"] = df["word_count"] - df[f"word_count_shift{gap}"]
            df[f"word_count_abs_change{gap}"] = np.abs(df[f"word_count_change{gap}"])
            
        df.drop(columns=[f"word_count_shift{gap}" for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        # [(特徴量の名前, [追加したい統計量])]
        feats_stat = [
            ("event_id", ["max"]),
            ("up_time", ["first", "last", "max", "mean", "std"]),
            ("down_time", ["first", "last", "max", "mean", "std"]),
            ("action_time", ["sum", "max", "mean", "std", "sem", skew, kurtosis]),
            ("activity", ["nunique"]),
            ("down_event", ["nunique"]),
            ("up_event", ["nunique"]),
            ("text_change", ["nunique"]),
            ("cursor_position", ["nunique", "max", "mean", "std"]),
            ("word_count", ["nunique", "max", "mean", "std"]),
        ]
    
        for gap in self.gaps:
            feats_stat.extend([
                (f"action_time_gap{gap}", ["first", "last", "max", "min", "mean", "std", "sum"]),
                (f"cursor_position_change{gap}", ["last", "max", "mean", "std", "sum"]),
                (f"word_count_change{gap}", ["mean", "std", "sum"]),
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                
                pbar.set_postfix(column=colname, method=method_name)
                grouped_df = df.groupby(["id"]).agg({colname: method}).reset_index().rename(
                    columns={colname: f"{colname}_{method_name}"}
                )
                feats = feats.merge(grouped_df, on="id", how="left")
        
        # input words
        tmp_df = self._calc_input_word_aggregations(df=df)
        feats = pd.merge(feats, tmp_df, on="id", how="left")
        
        # compare feats
        feats["word_time_ratio"] = feats["word_count_max"] / feats["up_time_max"]
        feats["word_event_ratio"] = feats["word_count_max"] / feats["event_id_max"]
        feats["event_time_ratio"] = feats["event_id_max"] / feats["up_time_max"]
        feats["idle_time_ratio"] = feats["action_time_gap1_sum"] / feats["up_time_max"]
        
        return feats
    
    def _count_essay_str(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """エッセイの文字数をカウント
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: (id, essay_len)のデータ
        """
        cols_to_drop = ["essay"]
        # Total essay length
        essay_df["essay_len"] = essay_df["essay"].apply(lambda v: len(v))
        essay_df = essay_df.drop(columns=cols_to_drop)
        
        return essay_df
    
    def _calc_word_aggregations(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """単語ごとに様々な統計量を計算
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: 単語ごとの統計量データ
        """
        word_df = self._split_essays_into_words(essay_df)
        
        word_agg_df = word_df[["id", "word_len"]].groupby(["id"]).agg([
            "mean",
            "std",
            "max",
            "first",
            "last",
            "sem",
            q1,
            "median",
            q3,
            "skew",
            pd.DataFrame.kurt,
            "sum",
        ])
        word_agg_df.columns = ["_".join(v) for v in word_agg_df.columns]
        word_agg_df["id"] = word_agg_df.index
        
        # 長さが以下のリスト内の要素を超える単語数の計算
        for v in [5, 6, 7, 8, 9, 10, 11, 12]:
            word_agg_df[f"word_len_ge_{v}_count"] = (
                word_df[word_df["word_len"] >= v].groupby(["id"]).count().iloc[:, 0]
            )
            word_agg_df[f"word_len_ge_{v}_count"] = word_agg_df[f"word_len_ge_{v}_count"].fillna(0)
            
        word_agg_df = word_agg_df.reset_index(drop=True)

        return word_agg_df
    
    def _split_essays_into_words(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """再構築したEssayを単語単位で分割
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: 単語単位に分割したEssayデータ
        """
        # 単語単位でsplitしたそれぞれをexplodeで連結
        essay_df["word"] = essay_df["essay"].apply(
            lambda v: re.split(" |\\n|\\n\\n|\\.|\\?|\\!", v)
        )
        essay_df = essay_df.explode("word")
        
        # 単語長(num of characters in word)
        essay_df["word_len"] = essay_df["word"].apply(lambda v: len(v))
        essay_df = essay_df[essay_df["word_len"] != 0]
        
        return essay_df
    
    def _calc_sentence_aggregations(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """文章ごとに様々な統計量を計算
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: 文章ごとの統計量データ
        """
        sentence_df = self._split_essays_into_sentences(essay_df)
        
        sentence_agg_df = sentence_df[
            ["id", "sentence_len", "sentence_word_count"]
        ].groupby(["id"]).agg([
            "mean",
            "std",
            "min",
            "max",
            "first",
            "last",
            "sem",
            q1,
            "median",
            q3,
            "skew",
            pd.DataFrame.kurt,
            "sum",
        ])
        
        sentence_agg_df.columns = ["_".join(v) for v in sentence_agg_df.columns]
        sentence_agg_df["id"] = sentence_agg_df.index
        
        # 長さが以下のリスト内の要素を超える文章数の計算
        for v in [50, 60, 75, 100]:
            sentence_agg_df[f"sentence_len_ge_{v}_count"] = (
                sentence_df[sentence_df["sentence_len"] >= v].groupby(["id"]).count().iloc[:, 0]
            )
            sentence_agg_df[f"sentence_len_ge_{v}_count"] = (
                sentence_agg_df[f"sentence_len_ge_{v}_count"].fillna(0)
            )
            
        sentence_agg_df = sentence_agg_df.reset_index(drop=True)
        
        return sentence_agg_df
    
    def _split_essays_into_sentences(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """再構築したEssayを文章単位で分割
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: 文章単位で分割したEssayデータ
        """
        # 文章単位でsplitしたそれぞれをexplodeで連結
        essay_df["sentence"] = essay_df["essay"].apply(
            lambda v: re.split("\\.|\\?|\\!", v)
        )
        essay_df = essay_df.explode("sentence")
        
        essay_df["sentence"] = essay_df["sentence"].apply(lambda v: v.replace("\n\n", "\n").replace("\n", "").strip())
        
        # 文章内の文字数
        essay_df["sentence_len"] = essay_df["sentence"].apply(lambda v: len(v))
        # 文章内の単語数
        essay_df["sentence_word_count"] = essay_df["sentence"].apply(lambda v: len(v.split(" ")))
        
        essay_df = essay_df[essay_df.sentence_len != 0].reset_index(drop=True)
        
        return essay_df
    
    def _calc_paragraph_aggregations(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """段落ごとに様々な統計量を計算
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: 段落ごとの統計量データ
        """
        paragraph_df = self._split_essays_into_paragraphs(essay_df)
        
        paragraph_agg_df = paragraph_df[
            ["id", "paragraph_len", "paragraph_word_count"]
        ].groupby("id").agg([
            "count",
            "mean",
            "std",
            "min",
            "max",
            "first",
            "last",
            "sem",
            q1,
            "median",
            q3,
            "skew",
            pd.DataFrame.kurt,
            "sum",
        ])
        
        paragraph_agg_df.columns = ["_".join(v) for v in paragraph_agg_df.columns]
        paragraph_agg_df["id"] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        
        return paragraph_agg_df
 
    def _split_essays_into_paragraphs(self, essay_df: pd.DataFrame) -> pd.DataFrame:
        """再構築したEssayを段落単位で分割
        
        Args:
            essay_df (pd.DataFrame): 再構築したEssayデータ
        Returns:
            pd.DataFrame: 段落単位で分割したEssayデータ
        """
        # 段落単位でsplitしたそれぞれをexplodeで連結
        essay_df["paragraph"] = essay_df["essay"].apply(
            lambda v: v.replace("\n\n", "\n").split("\n")
        )
        essay_df = essay_df.explode("paragraph")
        
        # 段落内の文字数
        essay_df["paragraph_len"] = essay_df["paragraph"].apply(lambda v: len(v))
        # 段落内の単語数
        essay_df["paragraph_word_count"] = essay_df["paragraph"].apply(lambda v: len(v.split(' ')))
        
        essay_df = essay_df[essay_df.paragraph_len != 0].reset_index(drop=True)
        
        return essay_df

    def _product_to_keys(self, df: pd.DataFrame, essay_df: pd.DataFrame) -> pd.DataFrame:
        """Essayの文字数に対するInput, Remove/Cutの割合を算出
        
        Args:
            df (pd.DataFrame): 入力データ
            essay_df (pd.DataFrame): Essayデータ
        Returns:
            pd.DataFrame: Essayの文字数に対するInput, Remove/Cutの割合
        """
        essay_df["product_len"] = essay_df["essay"].str.len()
        grouped_df = df[df["activity"].isin(["Input", "Remove/Cut"])].groupby("id").agg({"activity": "count"}).reset_index().rename(
            columns={"activity": "keys_pressed"}
        )
        
        output_df = essay_df.merge(grouped_df, on="id", how="left")
        output_df["product_to_keys"] = output_df["product_len"] / output_df["keys_pressed"]
        
        return output_df[["id", "product_to_keys"]]
    
    def _create_mouse_event_feature(self, df: pd.DataFrame) -> pd.DataFrame:
        """マウスを使ったイベントの特徴量を生成
        @ref: https://www.kaggle.com/code/crispychurch/keyboard-shortcuts-and-reliance-on-the-mouse/notebook
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: マウスを使ったイベントの特徴量データ
        """
        mouse_event_df = pd.DataFrame(df["id"].unique(), columns=["id"])
        
        mouse_event_df["mouse_event_cnt"] = df.groupby(df["id"])["down_event"].apply(
            lambda v: (v.isin(["Leftclick", "Rightclick", "Middleclick", "Unknownclick"]).sum())
        ).reset_index()["down_event"]
        
        all_event_cnt_df = df.groupby(df["id"])["event_id"].max().reset_index()["event_id"]
        
        mouse_event_df["mouse_event_ratio"] = (mouse_event_df["mouse_event_cnt"] / all_event_cnt_df)*100
        
        return mouse_event_df
    
    def _create_shortcut_event_feature(self, df: pd.DataFrame) -> pd.DataFrame:
        """ショートカットイベントの特徴量を生成
        @ref: https://www.kaggle.com/code/crispychurch/keyboard-shortcuts-and-reliance-on-the-mouse/notebook
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: ショートカットイベントの特徴量データ
        """
        event_df = df[["id", "event_id", "down_event"]].copy(deep=True)
        
        event_df["down_event_shift_1"] = event_df["down_event"].shift(periods=1)
        event_df["down_event_shift_2"] = event_df["down_event"].shift(periods=2)
        
        ctrl_c_df = (
            (event_df["down_event_shift_1"] == "Control") & (event_df["down_event"].str.lower() == "c")
        ).groupby(event_df["id"]).sum().reset_index(name="count")
        ctrl_v_df = (
            (event_df["down_event_shift_1"] == "Control") & (event_df["down_event"].str.lower() == "v")
        ).groupby(event_df["id"]).sum().reset_index(name="count")
        ctrl_x_df = (
            (event_df["down_event_shift_1"] == "Control") & (event_df["down_event"].str.lower() == "x")
        ).groupby(event_df["id"]).sum().reset_index(name="count")
        ctrl_left_df = (
            (event_df["down_event_shift_1"] == "Control") & (event_df["down_event"] == "ArrowLeft")
        ).groupby(event_df["id"]).sum().reset_index(name="count")
        
        output_df = pd.DataFrame(event_df["id"].unique(), columns=["id"])
        output_df["ctrl_c_cnt"] = ctrl_c_df["count"]
        output_df["ctrl_v_cnt"] = ctrl_v_df["count"]
        output_df["ctrl_x_cnt"] = ctrl_x_df["count"]
        output_df["ctrl_left_cnt"] = ctrl_left_df["count"]
        
        all_event_cnt_df = df.groupby(df["id"])["event_id"].max().reset_index()["event_id"]
        output_df["ctrl_c_ratio"] = (output_df["ctrl_c_cnt"] / all_event_cnt_df) * 100
        output_df["ctrl_v_ratio"] = (output_df["ctrl_v_cnt"] / all_event_cnt_df) * 100
        output_df["ctrl_x_ratio"] = (output_df["ctrl_x_cnt"] / all_event_cnt_df) * 100
        output_df["ctrl_left_ratio"] = (output_df["ctrl_left_cnt"] / all_event_cnt_df) * 100
        
        return output_df
    
    def _count_activity(self, df: pd.DataFrame) -> pd.DataFrame:
        """activityカラムのカウント
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: activityカラムを(tfidf|通常)でカウントしたデータ
        """
        grouped_df = df.groupby("id").agg({"activity": list}).reset_index()
        
        l = []
        for cols in tqdm(grouped_df["activity"].values):
            counter = list(Counter(cols).items())
            
            # 初期化
            d = {}
            for v in self.activities:
                d[v] = 0
                
            # "move from X to Y"も考慮する
            d["move_to"] = 0
            
            for v in counter:
                item, count = v[0], v[1]
                if item in d:
                    d[item] = count
                else:
                    d["move_to"] += count
                    
            l.append(d)
            
        count_df = pd.DataFrame(l)
        tfidf_df, normal_df = pd.DataFrame(l), pd.DataFrame(l)
        
        tfidf_cols = [f"activity_{v}_tfidf_count" for v in count_df.columns]
        normal_cols = [f"activity_{v}_normal_count" for v in count_df.columns]
        
        tfidf_df.columns = tfidf_cols
        normal_df.columns = normal_cols
        
        counts = tfidf_df.sum(1)
        for col in tfidf_cols:
            if col in self.idf.keys():
                idf =self.idf[col]
            else:
                idf = df.shape[0] / (tfidf_df[col].sum()+1)
                idf = np.log(idf)
                self.idf[col] = idf
                
            tfidf_df[col] = 1+np.log(tfidf_df[col] / counts)
            tfidf_df[col] *= idf
            
        agg_df = pd.concat([tfidf_df, normal_df], axis=1)
        
        return agg_df
    
    def _count_text_change(self, df: pd.DataFrame) -> pd.DataFrame:
        """text_changeカラムのカウント
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: text_changeカラムを(tfidf|通常)でカウントしたデータ
        """
        grouped_df = df.groupby("id").agg({"text_change": list}).reset_index()

        l = []
        for cols in tqdm(grouped_df["text_change"].values):
            counter = list(Counter(cols).items())
            
            # 初期化
            d = {}
            for k in self.text_changes_dict.keys():
                d[k] = 0
                
            for v in counter:
                item, count = v[0], v[1]
                if item in d:
                    d[item] = count
                    
            l.append(d)
            
        count_df = pd.DataFrame(l)
        tfidf_df, normal_df = pd.DataFrame(l), pd.DataFrame(l)
        
        tfidf_cols = [f"text_change_{self.text_changes_dict[v]}_tfidf_count" for v in count_df.columns]
        normal_cols = [f"text_change_{self.text_changes_dict[v]}_normal_count" for v in count_df.columns]
        
        tfidf_df.columns = tfidf_cols
        normal_df.columns = normal_cols
        
        counts = tfidf_df.sum(1)
        for col in tfidf_cols:
            if col in self.idf.keys():
                idf =self.idf[col]
            else:
                idf = df.shape[0] / (tfidf_df[col].sum()+1)
                idf = np.log(idf)
                self.idf[col] = idf
                
            tfidf_df[col] = 1+np.log(tfidf_df[col] / counts)
            tfidf_df[col] *= idf
            
        agg_df = pd.concat([tfidf_df, normal_df], axis=1)
        
        return agg_df
    
    def _count_punctuations(self, df: pd.DataFrame, colname: str) -> pd.DataFrame:
        """句読点の数をカウント
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: 句読点の数をカウントしたデータ
        """
        grouped_df = df.groupby("id").agg({colname: list}).reset_index()
            
        l = []
        for cols in tqdm(grouped_df[colname].values):
            c = 0
            counter = list(Counter(cols).items())
            for v in counter:
                item, count = v[0], v[1]
                if item in self.punctuations:
                    c += count
            l.append(c)
            
        count_df = pd.DataFrame({"punct_count": l})
        
        return count_df
    
    def _calc_p_bursts(self, df: pd.DataFrame) -> pd.DataFrame:
        """P-burstの計算
        一時停止するまでに書かれたもの = バーッと書いた際の行動？
        今回は2秒未満という条件にしている
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: P-burstの様々な統計量を含めたデータ
        """
        # up_timeを1行ずらして新しい列として追加
        df["up_time_lagged"] = df.groupby("id")["up_time"].shift(1).fillna(df["down_time"])

        # down_timeとup_time_laggedの差の絶対値を秒単位で計算し、新しい列として追加
        df["time_diff"] = abs(df["down_time"] - df["up_time_lagged"]) / 1000

        # 特定のactivityを持つ行をフィルタリング
        tmp_df = df[df["activity"].isin(["Input", "Remove/Cut"])].copy()

        # time_diffが2秒未満の場合にTrueを設定
        tmp_df["time_diff_lt_2"] = tmp_df["time_diff"] < 2

        # time_diff_lt_2がTrueの連続するグループごとにカウント
        tmp_df["P-bursts"] = tmp_df.groupby("id")["time_diff_lt_2"].transform(
            lambda x: x.cumsum() - x.cumsum().where(~x).ffill().fillna(0)
        )

        tmp_df = tmp_df.dropna(subset=["P-bursts"])
        
        # idごとに集計
        output_df = tmp_df.groupby("id").agg({"P-bursts": [
            "mean",
            "std",
            "max",
            "last",
            "sem",
            q1,
            "median",
            q3,
            "skew",
            pd.DataFrame.kurt,
            "sum",
        ]})

        # 列名の整形
        output_df.columns = ["_".join(col).strip() for col in output_df.columns.values]
        
        return output_df
    
    def _calc_r_bursts(self, df: pd.DataFrame) -> pd.DataFrame:
        """R-burstの計算
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: R-burstの様々な統計量を含めたデータ
        """
        # 'Input', 'Remove/Cut'のactivityを持つ行をフィルタリング
        tmp_df = df[df["activity"].isin(["Input", "Remove/Cut"])].copy()

        # 'Remove/Cut'のactivityを持つ行にTrueを設定
        tmp_df["activity_remove_cut"] = tmp_df["activity"] == "Remove/Cut"

        # activity_remove_cutがTrueの連続するグループごとにカウント
        tmp_df["R-bursts"] = tmp_df.groupby("id")["activity_remove_cut"].transform(
            lambda x: x.cumsum() - x.cumsum().where(~x).ffill().fillna(0)
        )

        tmp_df = tmp_df.dropna(subset=["R-bursts"])

        # idごとに集計
        output_df = tmp_df.groupby("id").agg({"R-bursts": [
            "mean",
            "std",
            "max",
            "sem",
            "skew",
            pd.DataFrame.kurt,
            "sum",
        ]})

        # 列名の整形
        output_df.columns = ["_".join(col).strip() for col in output_df.columns.values]

        return output_df
    
    def _calc_keys_pressed_per_second(self, df: pd.DataFrame) -> pd.DataFrame:
        """1sあたりのInput, Remove/Cutの割合を算出
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: Essayの文字数に対するInput, Remove/Cutの割合
        """
        grouped_df_cnt = df[df["activity"].isin(["Input", "Remove/Cut"])].groupby("id").agg(keys_pressed=("event_id", "count")).reset_index()
        grouped_df_diff = df.groupby("id").agg(min_down_time=("down_time", "min"), max_up_time=("up_time", "max")).reset_index()
        
        output_df = grouped_df_cnt.merge(grouped_df_diff, on="id", how="left")
        output_df["keys_per_second"] = output_df["keys_pressed"] / ((output_df["max_up_time"] - output_df["min_down_time"]) / 1000)
        
        return output_df[["id", "keys_per_second"]] 
    
    def _create_space_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """空白に関する特徴量を生成
        @ref: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs/notebook
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: 空白に関する特徴量データ
        """
        df["up_time_lagged"] = df.groupby("id")["up_time"].shift(1).fillna(df["down_time"])
        df["time_diff"] = abs(df["down_time"] - df["up_time_lagged"]) / 1000
        
        grouped_df = df.groupby("id")["time_diff"]
        largest_latency = grouped_df.max()
        median_latency = grouped_df.median()
        
        initial_pause = df.groupby("id")["down_time"].first() / 1000
        
        pause_half_sec = grouped_df.apply(lambda v: ((v >= 0.5)&(v < 1)).sum())
        pause_1_sec = grouped_df.apply(lambda v: ((v >= 1)&(v < 1.5)).sum())
        pause_1_half_sec = grouped_df.apply(lambda v: ((v >= 1.5)&(v < 2)).sum())
        pause_2_sec = grouped_df.apply(lambda v: ((v >= 2)&(v < 3)).sum())
        pause_3_sec = grouped_df.apply(lambda v: ((v >= 3).sum()))
        
        output_df = pd.DataFrame({
            "id": df["id"].unique(),
            "largest_latency": largest_latency,
            "median_latency": median_latency,
            "initial_pause": initial_pause,
            "pause_half_sec": pause_half_sec,
            "pause_1_sec": pause_1_sec,
            "pause_1_half_sec": pause_1_half_sec,
            "pause_2_sec": pause_2_sec,
            "pause_3_sec": pause_3_sec,
        }).reset_index(drop=True)
        
        return output_df
        
    def _calc_input_word_aggregations(self, df: pd.DataFrame) -> pd.DataFrame:
        """text_changeイベントで入力された単語の統計量を計算
        
        Args:
            df (pd.DataFrame): 入力データ
        Returns:
            pd.DataFrame: 入力された単語の統計量データ
        """
        tmp_df = df[
            (~df["text_change"].str.contains("=>")) & (df["text_change"] != "NoChange")
        ].reset_index(drop=True)
        
        grouped_df = tmp_df.groupby("id").agg({"text_change": list}).reset_index()
        
        # 整形
        grouped_df["text_change"] = grouped_df["text_change"].apply(lambda x: "".join(x))
        grouped_df["text_change"] = grouped_df["text_change"].apply(lambda x: re.findall(r"q+", x))
        
        # 統計量の算出
        grouped_df["input_word_count"] = grouped_df["text_change"].apply(len)
        grouped_df["input_word_len_mean"] = grouped_df["text_change"].apply(
            lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)
        )
        grouped_df["input_word_len_max"] = grouped_df["text_change"].apply(
            lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)
        )
        grouped_df["input_word_len_std"] = grouped_df["text_change"].apply(
            lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)
        )
        
        grouped_df.drop(["text_change"], axis=1, inplace=True)
        
        return grouped_df

In [12]:
preprocessor = Preprocessor(seed=Config.seed)

processed_train_df = preprocessor.make_feats(train_df)
processed_test_df = preprocessor.make_feats(test_df)

Starting to engineer features


  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


  0%|          | 0/2471 [00:00<?, ?it/s]

...

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


  0%|          | 0/3 [00:00<?, ?it/s]

> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 70
> for gap 100
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 70
> for gap 100
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 70
> for gap 100


  0%|          | 0/37 [00:00<?, ?it/s]

/tmp/ipykernel_20/3065027041.py:182: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats["word_time_ratio"] = feats["word_count_max"] / feats["up_time_max"]
/tmp/ipykernel_20/3065027041.py:183: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats["word_event_ratio"] = feats["word_count_max"] / feats["event_id_max"]
/tmp/ipykernel_20/3065027041.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at on

In [13]:
processed_train_df = processed_train_df.merge(scores_df, on="id", how="left")

In [14]:
processed_train_df.head()

,id,essay_len,word_len_mean,word_len_std,word_len_max,word_len_first,word_len_last,word_len_sem,word_len_q1,word_len_median,word_len_q3,word_len_skew,word_len_kurt,word_len_sum,word_len_ge_5_count,word_len_ge_6_count,word_len_ge_7_count,word_len_ge_8_count,word_len_ge_9_count,word_len_ge_10_count,word_len_ge_11_count,word_len_ge_12_count,sentence_len_mean,sentence_len_std,sentence_len_min,sentence_len_max,sentence_len_first,sentence_len_last,sentence_len_sem,sentence_len_q1,sentence_len_median,sentence_len_q3,sentence_len_skew,sentence_len_kurt,sentence_len_sum,sentence_word_count_mean,sentence_word_count_std,sentence_word_count_min,sentence_word_count_max,sentence_word_count_first,sentence_word_count_last,sentence_word_count_sem,sentence_word_count_q1,sentence_word_count_median,sentence_word_count_q3,sentence_word_count_skew,sentence_word_count_kurt,sentence_word_count_sum,sentence_len_ge_50_count,sentence_len_ge_60_count,sentence_len_ge_75_count,sentence_len_ge_100_count,paragraph_len_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,paragraph_len_q3,paragraph_len_skew,paragraph_len_kurt,paragraph_len_sum,paragraph_word_count_count,paragraph_word_count_mean,paragraph_word_count_std,paragraph_word_count_min,paragraph_word_count_max,paragraph_word_count_first,paragraph_word_count_last,paragraph_word_count_sem,paragraph_word_count_q1,paragraph_word_count_median,paragraph_word_count_q3,paragraph_word_count_skew,paragraph_word_count_kurt,paragraph_word_count_sum,product_to_keys,mouse_event_cnt,mouse_event_ratio,ctrl_c_cnt,ctrl_v_cnt,ctrl_x_cnt,ctrl_left_cnt,ctrl_c_ratio,ctrl_v_ratio,ctrl_x_ratio,ctrl_left_ratio,activity_Input_tfidf_count,activity_Remove/Cut_tfidf_count,activity_Nonproduction_tfidf_count,activity_Replace_tfidf_count,activity_Paste_tfidf_count,activity_move_to_tfidf_count,activity_Input_normal_count,activity_Remove/Cut_normal_count,activity_Nonproduction_normal_count,activity_Replace_normal_count,activity_Paste_normal_count,activity_move_to_normal_count,text_change_q_tfidf_count,text_change_space_tfidf_count,text_change_NoChange_tfidf_count,text_change_full_stop_tfidf_count,text_change_comma_tfidf_count,text_change_newline_tfidf_count,text_change_single_quote_tfidf_count,text_change_double_quote_tfidf_count,text_change_dash_tfidf_count,text_change_question_mark_tfidf_count,text_change_semicolon_tfidf_count,text_change_equals_tfidf_count,text_change_slash_tfidf_count,text_change_double_backslash_tfidf_count,text_change_colon_tfidf_count,text_change_q_normal_count,text_change_space_normal_count,text_change_NoChange_normal_count,text_change_full_stop_normal_count,text_change_comma_normal_count,text_change_newline_normal_count,text_change_single_quote_normal_count,text_change_double_quote_normal_count,text_change_dash_normal_count,text_change_question_mark_normal_count,text_change_semicolon_normal_count,text_change_equals_normal_count,text_change_slash_normal_count,text_change_double_backslash_normal_count,text_change_colon_normal_count,punct_count,P-bursts_mean,P-bursts_std,P-bursts_max,P-bursts_last,P-bursts_sem,P-bursts_q1,P-bursts_median,P-bursts_q3,P-bursts_skew,P-bursts_kurt,P-bursts_sum,R-bursts_mean,R-bursts_std,R-bursts_max,R-bursts_sem,R-bursts_skew,R-bursts_kurt,R-bursts_sum,keys_per_second,largest_latency,median_latency,initial_pause,pause_half_sec,pause_1_sec,pause_1_half_sec,pause_2_sec,pause_3_sec,event_id_max,up_time_first,up_time_last,up_time_max,up_time_mean,up_time_std,down_time_first,down_time_last,down_time_max,down_time_mean,down_time_std,action_time_sum,action_time_max,action_time_mean,action_time_std,action_time_sem,action_time_skew,action_time_kurtosis,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_mean,cursor_position_std,word_count_nunique,word_count_max,word_count_mean,word_count_std,a

## Check correlation coefficients

In [15]:
processed_train_df.drop(["id"], axis=1).corr().sort_values(["score"])["score"]

action_time_gap5_mean      -0.606004
action_time_gap10_mean     -0.602954
action_time_gap3_mean      -0.600881
action_time_gap20_mean     -0.596831
action_time_gap50_mean     -0.585539
                              ...   
sentence_len_ge_75_count    0.682049
word_len_ge_5_count         0.686487
word_len_ge_7_count         0.693650
word_len_ge_6_count         0.697498
score                       1.000000
Name: score, Length: 334, dtype: float64

## Split train data into folds for cross-validation

In [16]:
# kfold = KFold(n_splits=Config.num_folds, shuffle=True, random_state=Config.seed)

# for fold, (_, val_idx) in enumerate(kfold.split(processed_train_df)):
#     processed_train_df.loc[val_idx, "fold"] = fold

In [17]:
feature_names = list(
    filter(
        lambda x: x not in [Config.target_col, "id"], processed_train_df.columns
    )
)

In [18]:
X = processed_train_df.drop(columns=["id", Config.target_col], axis=1)
y = processed_train_df[Config.target_col].values

## Training

In [19]:
class ModelTrainer:
    def __init__(self, model_name: str, feature_names, X, y, **params):
        self.model_name = model_name
        
        self.feature_names = feature_names
        self.X = X
        self.y = y
        
        self.params = params

        self._create_model()
        
    def _make_pipeline(self, model):
        return Pipeline([
            ("remove_infs", FunctionTransformer(
                lambda v: np.nan_to_num(
                    v,
                    nan=np.nan,
                    posinf=0,
                    neginf=0,
                )
            )),
            ("imputer", SimpleImputer(strategy="mean")),
            ("normalizer", FunctionTransformer(lambda v: np.log1p(np.abs(v)))),
            ("scaler", RobustScaler()),
            ("model", model),
        ])
    
    def _create_model(self):
        match self.model_name:
            case "lgbm":
                self.model = LGBMRegressor(**self.params)
            case "xgb":
                self.model = self._make_pipeline(XGBRegressor(**self.params))
            case "catboost":
                self.model = CatBoostRegressor(**self.params)
            case "svr":
                self.model = self._make_pipeline(SVR(**self.params))
            case "ridge":
                self.model = self._make_pipeline(Ridge(**self.params))
            case 'rfr':
                self.model = self.make_pipeline(RandomForestRegressor(**self.params))
            case 'lasso':
                self.model = self.make_pipeline(Lasso(**self.params))
            case other:
                print("Not implemented")
                sys.exit(-1)
                
    def get_model(self):
        return self.model
    
    def train_model(self):
        skf = KFold(n_splits=Config.num_folds, random_state=Config.seed, shuffle=True)
        
        fold_rmses = []
        for fold, (train_idx, val_idx) in enumerate(skf.split(self.X, self.y)):
            print(f"Training for Fold {fold}")

            X_train = self.X.iloc[train_idx][self.feature_names]
            y_train = self.y[train_idx]

            X_val = self.X.iloc[val_idx][self.feature_names]
            y_val = self.y[val_idx]

            if self.model_name == "lgbm":
                early_stopping_callback = lgbm.early_stopping(
                    Config.early_stopping_round,
                    first_metric_only=True,
                    verbose=Config.verbose,
                )
                
                self.model.fit(
                    X_train,
                    y_train,
                    eval_set=[(X_val, y_val)],
                    eval_metric=Config.metric,
                    callbacks=[lgbm.early_stopping(stopping_rounds=Config.early_stopping_round)]
                )
            else:
                self.model.fit(X_train, y_train)
                
            preds = self.model.predict(X_val)
            rmse = mean_squared_error(y_true=y_val, y_pred=preds, squared=False)
            
            fold_rmses.append(rmse)
            
        avg_rmse = np.mean(fold_rmses)
        print(f"Average rmse: {avg_rmse}")
        
        return avg_rmse
            
    def evaluate(self):
        """全データに対して性能評価"""
        preds = self.predict(self.X)
        
        rmse = mean_squared_error(y_true=self.y, y_pred=preds, squared=False)
        
        return rmse
    
    def predict(self, X_test):
        """X_testに対する推論"""
        y_test = np.zeros((len(X_test), Config.num_folds))
        
        for fold in range(Config.num_folds):
            preds = self.model.predict(X_test)
            y_test[:, fold] = preds
        
        y_test = np.mean(y_test, axis=1)
        
        return y_test
    
    def clear_memory(self):
        del self.model
        
        libc.malloc_trim(0)
        torch.cuda.empty_cache()
        gc.collect()

### Optimize Hyperparameters with Optuna

In [20]:
params = {}

params["lgbm"] = {
    'metric': Config.metric, 
    'random_state': Config.seed,
    'n_estimators': Config.num_boost_rounds,
    'reg_alpha': 0.061765887407558166,
    'reg_lambda': 7.55770932741589,
    'colsample_bytree': 0.5134789473998289,
    'subsample': 0.9736470828995958,
    'learning_rate': 0.08342880778233056,
    'num_leaves': 9,
    'min_child_samples': 100,
}

params["xgb"] = {
    "objective": "reg:squarederror",
    "n_estimators": 1024,
    # pipelineにeval_setの引数をわたせる形にする必要あり
#     "early_stopping_rounds": Config.early_stopping_round,
    "eval_metric": Config.metric,
    "seed": Config.seed,
    "max_depth": 4,
    "learning_rate": 0.1,
}

params["catboost"] = {
    "iterations": Config.num_boost_rounds // 10,
    "early_stopping_rounds": Config.early_stopping_round,
    "loss_function": Config.metric.upper(),
    "random_seed": Config.seed,
    "silent": True,
    "depth": 6,
}

params["svr"] = {
    "kernel": "rbf",
    "C": 1.0,
    "epsilon": 0.1,
}

params["ridge"] = {
    "solver": "auto",
    "random_state": Config.seed,
    "alpha": 1,
}

params["rfr"] = {
    'max_depth': 6,
    'max_features': 'sqrt',
    'min_impurity_decrease': 0.0016295128631816343,
    'n_estimators': 200,
    'random_state': Config.seed,  
}

params["lasso"] = {
    'alpha': 0.04198227921905038, 
    'max_iter': 2000, 
    'random_state': Config.seed,
}

In [21]:
best_score = 1.0
def objective(trial, model_name, feature_names, X, y):
    global params
    d = params[model_name]
    if model_name == "lgbm":
        d['reg_alpha'] = trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True)
        d['reg_lambda'] = trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True)
        d['colsample_bytree'] = trial.suggest_float('colsample_bytree', 0.5, 1)
        d['subsample'] = trial.suggest_float('subsample', 0.5, 1)
        d['learning_rate'] = trial.suggest_float('learning_rate', 1e-4, 0.1, log=True)
        d['num_leaves'] = trial.suggest_int('num_leaves', 8, 64)
        d['min_child_samples'] = trial.suggest_int('min_child_samples', 1, 100)
    elif model_name == "xgb":
        d["max_depth"] = trial.suggest_int("max_depth", 2, 64)
        d["learning_rate"] = trial.suggest_loguniform("learning_rate", 1e-4, 0.5)
    elif model_name == "catboost":
        d["depth"] = trial.suggest_int("depth", 2, 30)
    elif model_name == "svr":
        d["epsilon"] = trial.suggest_float("epsilon", 0.01, 1)
    elif model_name == "ridge":
        d["alpha"] = trial.suggest_loguniform("alpha", 1e-3, 10.0)
    # TODO: rfr, lasso

    # Experiment the params
    trainer = ModelTrainer(model_name, feature_names, X, y, **d)
    avg_score = trainer.train_model()
    
    # best_scoreの更新
    global best_score
    if avg_score < best_score:
        best_score = avg_score
        
    trainer.clear_memory()
    del trainer
    
    print(f"Average result {avg_score} and the best score {best_score}")
    
    return avg_score

def run_optuna(model_name, feature_names, X, y):
    study = optuna.create_study(direction="minimize", study_name=f"{model_name}_study")
    study.optimize(
        lambda trial: objective(trial, model_name, feature_names, X, y),
        n_trials=Config.num_trials_optuna,
        show_progress_bar=True,
        gc_after_trial=True,
    )
    
    best_trial = study.best_trial
    best_params = study.best_params
        
    return study

### Train(Ensemble)

In [22]:
def train_model(model_name, feature_names, X, y):
    best_params = params[model_name]
    if Config.USES_OPTUNA_TUNING:
        study = run_optuna(model_name, feature_names, X, y)
        best_params.update(study.best_params)
    
    trainer = ModelTrainer(model_name, feature_names, X, y, **best_params)
    
    trainer.train_model()
    rmse = trainer.evaluate()
    
    print(f"Best params: {best_params}")
    print(f"Complete trianing {model_name} RMSE={rmse}")
    
    return trainer.get_model()

In [23]:
models = []
model_names = ["lgbm", "xgb", "catboost", "svr", "ridge"]

y_preds = []
y_test = []
for v in model_names:
    model = train_model(v, feature_names, X, y)
    models.append((v, model))

print(models)

Training for Fold 0
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[51]	valid_0's rmse: 0.573061
Training for Fold 1
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[175]	valid_0's rmse: 0.519062
Training for Fold 2
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[100]	valid_0's rmse: 0.692795
Training for Fold 3
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[129]	valid_0's rmse: 0.611135
Training for Fold 4
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[78]	valid_0's rmse: 0.577221
Training for Fold 5
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is:
[56]	valid_0's rmse: 0.624402
Training for Fold 6
Training until validation scores don't improve for 250 rounds
Early stopping, best iteration is

## Evaluate

### Use VotingRegressor Class

- モデルの予測を組み合わせる
- モデルの重みは、アンサンブルの予測への最適な寄与を保証するため完全なトレーニングデータセットでのRMSEスコアによって決定

In [24]:
# 全データを用いてアンサンブルの割合を出すので、CVがもう使えない
def evaluate_models(models, X, y):
    # 全データに対して、split
    X_train, X_val, y_train, y_val = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=Config.seed,
    )
    
    # fit and evaluate
    weights = list()
    for name, model in models:
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        
        rmse = mean_squared_error(y_true=y_val, y_pred=preds, squared=False)
        
        # RMSEの逆数を用いる
        weights.append(1/rmse)
    
    print(f"Weight = {weights}")
    
    return weights

In [25]:
try:
    weights = evaluate_models(models, X, y)
    # 正規化
    weights = [w / sum(weights) for w in weights]
    
    # SkLearnのVotingRegressorを用いてアンサンブル
    ensemble = VotingRegressor(estimators=models, weights=weights)
    ensemble.fit(X, y)
except Exception as e:
    print(e)

Weight = [1.7521480330239048, 1.7577723915673873, 1.8213961485982257, 1.7376985181108024, 1.7657802900989967]


In [26]:
ensemble

VotingRegressor(estimators=[('lgbm',
                             LGBMRegressor(colsample_bytree=0.5134789473998289,
                                           learning_rate=0.08342880778233056,
                                           metric='rmse', min_child_samples=100,
                                           n_estimators=50500, num_leaves=9,
                                           random_state=42,
                                           reg_alpha=0.061765887407558166,
                                           reg_lambda=7.55770932741589,
                                           subsample=0.9736470828995958)),
                            ('xgb',
                             Pipeline(steps=[('remove_infs',
                                              FunctionTransforme...
                                              FunctionTransformer(func=<function ModelTrainer._make_pipeline.<locals>.<lambda> at 0x799ba804f640>)),
                                             ('imputer', SimpleImputer()),
                                             ('normalizer',
                                              FunctionTransformer(func=<function ModelTrainer._make_pipeline.<locals>.<lambda> at 0x799ba804feb0>)),
                                             ('scaler', RobustScaler()),
                                             ('model',
                                              Ridge(alpha=1,
                                                    random_state=42))]))],
                weights=[0.19832355559845316, 0.1989601700644004,
                         0.20616166758462495, 0.19668803216080524,
                         0.19986657459171625])

## Make submission

In [27]:
# predict
X_test = processed_test_df[feature_names]
preds = ensemble.predict(X_test)
processed_test_df[Config.target_col] = preds

processed_test_df[Config.target_col] = np.clip(
    processed_test_df[Config.target_col],
    a_min=0.0,
    a_max=6.0,
)

submission_df = processed_test_df[["id", Config.target_col]]

submission_df

/tmp/ipykernel_20/2963647557.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  processed_test_df[Config.target_col] = preds


,id,score
0,0000aaaa,0.0
1,2222bbbb,0.0
2,4444cccc,0.0


In [28]:
submission_df.to_csv('submission.csv', index=False)

## Reference

- https://www.kaggle.com/code/takaito/lwpwq-public-baseline-notebook
    - v1(0.659)
- https://www.kaggle.com/code/mcpenguin/writing-processes-to-quality-baseline
    - v2()